# UK Orders by City and Competitor Locations (20 mile radius)

**Project Description:**


This visualization is tailored for a London-based retailer looking to grow their stores across the UK. It features a map that shows where the retailer's online orders are coming from, helping to spot the best cities for new stores. The map also highlights where the top three competitors are located to avoid stepping into highly competitive areas. There’s a handy toggle in the top right corner of the map, so you can easily show or hide these competitors' locations, making it simpler to see where the next store could thrive.

In [12]:
import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
from folium.plugins import BeautifyIcon
from folium.plugins import MiniMap
import numpy as np
from geopy.distance import great_circle
from folium.map import Popup
from folium import DivIcon

data=pd.read_csv('UK Orders by City.csv')

stores=pd.read_csv('Competitor Locations.csv')

# Create a Folium map centered on the UK
map_center = [55, -4.5]  
my_map = folium.Map(location=map_center, zoom_start=6)


# Define a color mapping for each competitor store chain
color_mapping = {
    'Competitor 1': 'blue',
    'Competitor 2': 'orange',
    'Competitor 3':'red',
}

legend_entries = {}

feature_groups = {channel: folium.FeatureGroup(name=channel) for channel in color_mapping}

# Create marker and circle layers for each store
for index, row in pd.DataFrame(stores).iterrows():
    location = (row['lat'], row['long'])
    channel = row['CHANNEL']

    marker_color = color_mapping.get(channel, 'gray')

    # Create a 20-mile radius around competitor stores
    circle = folium.Circle(
        location=location,
        radius=10 * 1609.34,  
        color=marker_color,
        fill=True,
        fill_opacity=0.1,  
        weight=1,  
    )
    
    circle.add_to(feature_groups[channel])

for channel, feature_group in feature_groups.items():
    feature_group.add_to(my_map)

for index, row in pd.DataFrame(stores).iterrows():
    location = (row['lat'], row['long'])
    channel = row['CHANNEL']

    marker_color = color_mapping.get(channel, 'gray')

    marker = folium.CircleMarker(
        location=location,
        radius=3, 
        color=marker_color,
        fill=True,
        fill_opacity=0.7)
    
# Function to create markers based on transactions metric by city
def create_marker(row, size_column, tooltip_column, color):
    location = (row['ga:latitude'], row['ga:longitude'])
    marker_size = row[size_column]

    # Normalize marker size logarithmically for better differentiation
    normalized_size = 3 + 10 * ((marker_size - data['ga:transactions'].min()) / (data['ga:transactions'].max() - data['ga:transactions'].min())) * 1.25

    marker = folium.CircleMarker(
        location=location,
        radius=normalized_size,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.5,
        tooltip=f"{row[tooltip_column]} - {marker_size}"  # Tooltip with city name and metric value
    )

    return marker

# Function to add markers based on transactions

def add_markers_for_transactions(color):
    tooltip_column = 'ga:city'

    marker_group = folium.FeatureGroup(name='transactions')
    for index, row in data.iterrows():
        marker = create_marker(row, 'ga:transactions', tooltip_column, color)
        marker_group.add_child(marker)

    my_map.add_child(marker_group)

# Add markers for transactions metric with green color
add_markers_for_transactions('green')

# Create a LayerControl to toggle between Competitors
layer_control = folium.LayerControl()
layer_control.add_to(my_map)

# Add title
title_html = """
<div style="position: fixed; top: 10px; left: 50%; transform: translateX(-50%); background-color: white;
    border: 2px solid grey; z-index: 9999; padding: 10px; font-size: 16px;">
    <b>UK Orders by City and Competitor Locations (20 mile radius)</b>
</div>
"""
my_map.get_root().html.add_child(folium.Element(title_html))


# Function to generate HTML for legend
def create_legend(color_mapping):
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 70px; z-index: 1000; padding: 10px;
        background-color: white; border: 2px solid grey; border-radius: 5px; font-size: 12px;">
        <p><b>Legend</b></p>
    """

    for channel, color in color_mapping.items():
        legend_html += f'<p><i class="fa fa-circle" style="color:{color};"></i> {channel}</p>'

    legend_html += '<p><i class="fa fa-circle" style="color:green;"></i> Transactions</p>'

    legend_html += "</div>"
    return legend_html

# Create & Add the legend HTML
legend_html = create_legend(color_mapping)

my_map.get_root().html.add_child(folium.Element(legend_html))


# Show the map
my_map

# Save the map
#my_map.save("UK Orders by City and Competitor Locations (20 mile radius).html")